In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service # Import Service class to specify ChromeDriver location
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
import pandas as pd

In [47]:
# Replace with the path to your ChromeDriver executable
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-sh-usage")
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.amazon.com/')
# Open Facebook login page

In [48]:
df = pd.read_csv('reviewers_links/mens_clothes_reviewers.csv')

links = df['links']

# https://www.amazon.com/gp/profile/amzn1.account.AHNMBMZ6VIRUY6MRF47PSUP4W34Q/ref=cm_cr_arp_d_gw_btm?ie=UTF8
ids = df['reviewer_id']
    
len(links), len(ids)

(5045, 5045)

In [62]:
data = []
cat = "Clothing, Shoes & Jewelry"
for i, link in enumerate(links):
    if i < 10 and i < len(links):
        driver.get(link)
        time.sleep(1)
        main_reviews_div = driver.find_element(By.ID, "profile-at-card-container")
        all_reviews_div = main_reviews_div.find_elements(
            By.CLASS_NAME, "your-content-card-wrapper"
        )
        # print(f'len: {len(all_reviews_div)}')
        for review in all_reviews_div:
            try:
                div = review.find_element(By.CLASS_NAME, "your-content-card-reactions")
                anchor = review.find_element(By.TAG_NAME, "a")
                span = div.find_element(By.TAG_NAME, "span").text
                if int(span) > 9:
                    a = anchor.get_attribute("href")
                    print(a)
                    data.append(a)
                    driver.get(a)
                    time.sleep(1)
                    product_div = driver.find_element(
                        By.CSS_SELECTOR, ".a-row.a-spacing-top-mini.a-text-ellipsis"
                    )
                    product_url = product_div.find_element(
                        By.TAG_NAME, "a"
                    ).get_attribute("href")
                    driver.get(product_url)
                    time.sleep(1)
                    category_div = driver.find_element(
                        By.ID, "wayfinding-breadcrumbs_feature_div"
                    )
                    category = category_div.find_element(By.TAG_NAME, "a").text
                    #######################
                    # print('1')
                    #######################
                    if cat not in category:
                        driver.back()
                    else:
                        #######################
                        print('entered else')
                        #######################
                        product_title = driver.find_element(By.ID, "productTitle").text
                        #######################
                        print(f'title received: {product_title}')
                        #######################
                        average_rating = float(
                            driver.find_element(By.ID, "acrPopover")
                            .get_attribute("title")
                            .split(" ")[0]
                        )
                        #######################
                        print(f'average rating received: {average_rating}')
                        #######################
                        total_rating = int(
                            driver.find_element(By.ID, "acrCustomerReviewText")
                            .text.split(" ")[0]
                            .replace(",", "")
                        )
                        #######################
                        print(f'total rating received: {total_rating}')
                        #######################
                        categories_div = driver.find_elements(
                            By.CSS_SELECTOR,
                            ".a-unordered-list.a-horizontal.a-size-small > li:not([class])",
                        )
                        #######################
                        print(f'categories div received: {len(categories_div)}')
                        #######################
                        product_category = []
                        for el in categories_div:
                            anc = el.find_element(By.TAG_NAME, "a").text
                            #######################
                            print(f'anchor found in categories div: {anc}')
                            #######################
                            product_category.append(anc)
                        product_category = ", ".join(product_category)
                        #######################
                        print(f'product_category: {product_category}')
                        #######################
                        details_div = driver.find_element(
                            By.CSS_SELECTOR,
                            ".a-expander-content.a-expander-partial-collapse-content.a-expander-content-expanded",
                        )
                        #######################
                        print(f'details div received')
                        #######################
                        feature_divs = details_div.find_elements(By.CSS_SELECTOR, '.a-fixed-left-grid.product-facts-detail')
                        #######################
                        print(f'feature div received: {lrn(feature_divs)}')
                        #######################
                        features = {}
                        for feat in feature_divs:
                            feat_head = feat.find_element(By.CSS_SELECTOR, '.a-fixed-left-grid-col.a-col-left')
                            feat_name = feat_head.find_element(By.CLASS_NAME, 'a-color-base').text
                            #######################
                            print(f'feat head received: {feat_name}')
                            #######################
                            feat_tail = feat.find_element(By.CSS_SELECTOR, '.a-fixed-left-grid-col.a-col-right')
                            feat_value = feat_tail.find_element(By.CLASS_NAME, 'a-color-base').text
                            #######################
                            print(f'feat value received: {feat_value}')
                            #######################
                            features[feat_name] = feat_value
                        desc_more = details_div.find_element(By.ID, 'productFactsToggleButton').find_element(By.TAG_NAME, 'a')
                        #######################
                        print('more description found')
                        #######################
                        desc_more.click()
                        #######################
                        print('more description clicked')
                        #######################
                        desc_div = details_div.find_elements(By.CSS_SELECTOR, '.a-unordered-list.a-vertical.a-spacing-small')
                        #######################
                        print(f'description div received: {len(desc_div)}')
                        #######################
                        description = []
                        for el in desc_div:
                            li = el.find_element(By.TAG_NAME, 'li')
                            #######################
                            print('got list item')
                            #######################
                            desc = li.find_element(By.TAG_NAME, 'span').text
                            #######################
                            print(f'got desctiption: {desc}')
                            #######################
                            description.append(desc)
                        description = (", ").join(description)
                        product_asin = driver.find_element(By.XPATH, "//div[@id='detailBullets_feature_div']//span[contains(text(), 'ASIN')]/following-sibling::span").text
                        #######################
                        print(f'got product asin: {product_asin}')
                        #######################
                        
                        driver.back()
                        #######################
                        print('get back')
                        #######################
                        time.sleep(1)
                        user_name = driver.find_element(By.CLASS_NAME, 'a-profile-name').text
                        #######################
                        print(f'user name received: {user_name}')
                        #######################
                        review_title = driver.find_element(By.CSS_SELECTOR, "a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold > span:not([class])").text
                        #######################
                        print(f'review title reveived: {review_title}')
                        #######################
                        verified_purchase = True if driver.find_element(By.CSS_SELECTOR, '.a-size-mini.a-color-state.a-text-bold').text == 'Verified Purchase' else False
                        #######################
                        print(f'verified purchase received: {verified_purchase}')
                        #######################
                        review_rating = float(driver.find_element(By.CSS_SELECTOR, "a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold > i > span").text.split(' ')[0])
                        #######################
                        print(f'review rating received: {review_rating}')
                        #######################
                        text_span = driver.find_element(By.CSS_SELECTOR, '.a-size-base.review-text.review-text-content')
                        #######################
                        print('got review body div')
                        #######################
                        review_text = text_span.find_element(By.TAG_NAME, 'span').text.replace('<br>', ' ')
                        #######################
                        print(f'review body received: {review_text}')
                        #######################
                        helpful_votes = driver.find_element(By.CSS_SELECTOR, '.a-size-base.a-color-tertiary.cr-vote-text').text.split(' ')[0]
                        #######################
                        print(f'got helpful votes: {helpful_votes}')
                        #######################
                        helpful_votes = 1 if helpful_votes == 'One' else int(helpful_votes)
                        
                        row = {
                            'product_title': product_title,
                            'average_rating': average_rating,
                            'total_rating': total_rating,
                            'product_category': product_category,
                            'features': features,
                            'description': description,
                            'product_asin': product_asin,
                            'user_name': user_name,
                            'review_title': review_title,
                            'verified_purchase': verified_purchase,
                            'review_rating': review_rating,
                            'text': review_text,
                            'helpful_votes': helpful_votes,
                            'user_id': link.split('/ref')[0].split('account.')[1]
                        }
                        data.append(row)
                        time.sleep(1)
                        print(row)
                    driver.back()
            except:
                continue
        print(f"{i} : reviewers done")

0 : reviewers done
1 : reviewers done
https://www.amazon.com/gp/customer-reviews/R1Q9IMHUIXCMGR?ref=pf_vv_at_pdctrvw_srp
2 : reviewers done
3 : reviewers done
4 : reviewers done
5 : reviewers done
6 : reviewers done
7 : reviewers done
8 : reviewers done
https://www.amazon.com/gp/customer-reviews/R2Q3AU93E5T46O?ref=pf_vv_at_pdctrvw_srp
entered else
title received: Match Men's Slim Tapered Stretchy Casual Pants
average rating received: 4.3
total rating received: 6231
categories div received: 5
anchor found in categories div: Clothing, Shoes & Jewelry
anchor found in categories div: Men
anchor found in categories div: Clothing
anchor found in categories div: Pants
anchor found in categories div: Casual
product_category: Clothing, Shoes & Jewelry, Men, Clothing, Pants, Casual
9 : reviewers done


In [51]:
len(list(set(data)))

2